In [105]:
import praw
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from functools import lru_cache
from collections import deque

In [26]:
nltk.download('vader_lexicon')
analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/lingxiaowang/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [11]:
reddit = praw.Reddit(
    client_id=,
    client_secret= ,
    password= ,
    user_agent= ,
    username= ,
)

In [20]:
submission = reddit.submission(id="r1uqk0")
submission.comments.replace_more(limit=None)
comments = submission.comments.list()

In [187]:
print(comments[10].body)
analyzer.polarity_scores(comments[116].body)

“I guess we live in a world now that our public schools would rather have kids read about gay pornography than Christ.”

Public school. 

PUBLIC SCHOOL. 

Go to your non-secular private school if you want religious education. Unless you want to advocate for teaching about every major religion, then religion has no place in public education. Something tells me the kind of person complaining that the system isn’t Christian enough isn’t concerned with balanced and diverse education.


{'neg': 0.119, 'neu': 0.881, 'pos': 0.0, 'compound': -0.4019}

In [190]:
for comment in comments:
    if analyzer.polarity_scores(comment.body)["compound"]>0.7:
        print(comment.body)
        print(analyzer.polarity_scores(comment.body))

A saying that I first heard only this year: “People don’t want freedom; they want a dictator who agrees with them”
{'neg': 0.0, 'neu': 0.619, 'pos': 0.381, 'compound': 0.765}
Brought to you by the party of "small government" that never shuts the fuck up about "muh freedoms!"
{'neg': 0.0, 'neu': 0.733, 'pos': 0.267, 'compound': 0.7043}
Yes, they’re trying to do that here in FL.  It’s ridiculous.  “Moms for Liberty,” lol.  Clearly they don’t support having actual liberty. 🙄
{'neg': 0.076, 'neu': 0.488, 'pos': 0.436, 'compound': 0.8957}
" just a coincidence that most of the authors were black" wow that's rich

Also, our seems to me a lot of religious texts including the bible are very graphic. I'll bet if we turned the tables and started banning or burning them they'd suddenly be all pro free speech...
{'neg': 0.0, 'neu': 0.804, 'pos': 0.196, 'compound': 0.8934}
The American Library Association Office for Intellectual Freedom provides confidential support during censorship challenges to l

{'neg': 0.086, 'neu': 0.777, 'pos': 0.136, 'compound': 0.8334}
You read this article like it’s something conservatives actually want. Every group of any any party or religion has people that go too far. I’m conservative and my family is. We all read these books as a kid in Catholic school. I had a class where we read lots of throngs and watched the movie and talked about and read dune tones of cool books like that. We read to kill a mockingbird. We also read the one with the guy that was so strong and dumb his friend had to kill him. Forgot the name. These books are important to history and very well made. My family is conservative too. Literally no one supports banning of books. It’s shitty. There’s religious extremists out there that say everything is of the devil, I’m Catholic. I don’t listen to those idiots. 

We also read a very very important book. Fahrenheit 451. Which is literally about topics like this and how important it is to have the freedom to read books. 

Your picking o

Well that part is pretty much true. The difference in reality is that voters can create dramatic change through the primaries. If they only show up for the general election, they have already thrown that chance away.
{'neg': 0.0, 'neu': 0.724, 'pos': 0.276, 'compound': 0.8807}
Yeah...it's super bizarre, equating retelling a childhood story about yourself and pedophilia. Like...they were both kids. And the person speaking about it was involved in it. Is talking about your first kiss pedophilia? Is telling people how you lost your virginity pedophilia? 

But the big thing to remember is situations like this is logic will NEVER change their mind. Because they're living based on "faith" AKA: mindless brainwashing. The concept of faith directly contradicts rational thought. Questions make you smarter, so if you demonize questions you get to control a bunch of dumbasses you manufactured yourself!! Huzzah!!!

I love how people always ask "without religion how would we keep people from murderi

I think you'd be surprised, there's a lot of places where their copy has been stolen or destroyed and it's not replaced, but it's not actually banned. Banning and censoring books goes against the core values of most people on the left. It's okay to point out the hypocrisy of selective banning by the right, to suit their needs, but I'd prefer if nothing was banned from my kid's school. If you want someone to realize how boring and stupid Nazis are, hand them a copy of Mein Kampf and tell them to read it cover to cover. The same applies to Christianity and the Bible respectively.
{'neg': 0.093, 'neu': 0.728, 'pos': 0.179, 'compound': 0.8717}
True, but in the context of this conversation/article.. I'd say a school library is probably a pretty safe and educational environment for kids to read about tough topics like sex or racism.. with resources like teachers to come to with questions
{'neg': 0.034, 'neu': 0.643, 'pos': 0.323, 'compound': 0.9413}
That’s fair, but also I don’t love the lac

In [198]:
class Node:
    def __init__(self, comment = None):
        self.comment = comment
        #self.user = comment.author
        self.receiver = set()
    
    @lru_cache(maxsize=None)
    def get_score(self):
        score = analyzer.polarity_scores(self.comment.body)
        return score

def to_node(comment, depth, max_depth):
    # convert from comment to tree
    node = Node(comment)
    
    if depth >= max_depth:
        return node
    
    replies = comment.replies
    for reply in replies:
        rep_node = to_node(reply, depth+1, max_depth)
        node.receiver.add(rep_node)
    return node
    
def ls_to_csv(ls, name_str):
    # Convert from list to .csv file adaptable to statnet attributes
    ls_dic = {name_str: ls}
    df = pd.DataFrame(ls_dic)
    df.to_csv(name_str+'.csv', index = False)
    
def print_csv(head):
    # print the tree with Node as head
    queue = deque()
    queue.append((head, 1))
    Sender, Receiver = [], []
    Neg, Neu, Pos, Compound = [], [], [], []
    is_pos, is_neg = [], []
    length = []
    upvote = []
    upvote_thres = []
    is_extreme = []
    bfs_index = 0
    while queue:
        node, sender_id = queue.popleft()
        bfs_index += 1
        if bfs_index > 1:
            Sender.append(sender_id)
            Receiver.append(bfs_index)
        
        score = node.get_score()
        Neg.append(score['neg'])
        Neu.append(score['neu'])
        Pos.append(score['pos'])
        upvote.append(node.comment.score)
        if node.comment.score > 500:
            upvote_thres.append(1)
        else:
            upvote_thres.append(0)
        
        
        length.append(len(node.comment.body))
        
        if score['compound']>0:
            is_pos.append(1)
            is_neg.append(0)
        else:
            is_pos.append(0)
            is_neg.append(1)
        
        
        Compound.append(score['compound'])
        
        if abs(score['compound'] > 0.7):
            is_extreme.append(1)
        else:
            is_extreme.append(0)
        
        for rec in node.receiver:
            queue.append((rec, bfs_index))
    
    link_dic = {'Sender': Sender, 'Receiver':Receiver}
    link_df = pd.DataFrame(link_dic)
    link_df.to_csv('link.csv', index = False)
    
    
    ls_to_csv(Neg, 'neg')
    ls_to_csv(Neu, 'neu')
    ls_to_csv(Pos, 'pos')
    ls_to_csv(Compound, 'compound')
    ls_to_csv(is_pos, 'is_pos')
    ls_to_csv(is_neg, 'is_neg')
    ls_to_csv(is_extreme, 'is_extreme')
    ls_to_csv(length, 'length')
    ls_to_csv(upvote, 'upvote')
    ls_to_csv(upvote_thres, 'upvote_thres')

In [199]:
depth = 0
max_depth = 4
head = to_node(comments[0], depth, max_depth)
print_csv(head)